## OGC Web Coverage Service (WCS) 2.0.1

This notebook gives a short introduction to the Open Geospatial Consortium (OGC) standard WCS and WCPS (Web Coverage Processing Service). Small examples help you to get started retrieving data from a WCS server.

___________________________________________________________________________________ 

### OGC WCS Core Requests

OGC Web Coverage Service (v.2.0.1) is a standarized data access protocol. 
There are three core requests in order to get access to data.

The WCS server at ECMWF can be accessed via the url below - by specifying either one of the core requests or the WCS extension.<br> 
http://incubator.ecmwf.int/2e/rasdaman/ows?service=WCS&version=2.0.1&request=...

* **GetCapabilities**
Returns a XML-encoded description of service properties and all data (coverages) available on the server.<br>
    * The **"Capabilities document"** is returned and contains information e.g. service provider, to format encodings, supported interpolation methods, coverages on the server <br>

    Example: http://incubator.ecmwf.int/2e/rasdaman/ows?service=WCS&version=2.0.1&request=GetCapabilities


* **Describe Coverage**
Returns XML-encoded description of a specific coverage, to check resolution, dimension etc. <br>
    * Returns the **"Coverage description document"** that contains information to spatial and temporal dimension and general information to the type of coverage <br>

    Example: http://incubator.ecmwf.int/2e/rasdaman/ows?service=WCS&version=2.0.1&request=DescribeCoverage&coverageId=T2m


* **GetCoverage**
Returns a coverage or spatial / temporal subset of the coverage --> allows slicing and trimming <br>
    * Returns either an xml-encoded file with the resulting information as a tuple list

    Example: Retrieve 2m air temperature for the year 2012 for Lat 52 - 54 and Lon 0 - 2 <br>
        http://incubator.ecmwf.int/2e/rasdaman/ows?service=WCS&version=2.0.1&request=GetCoverage&coverageId=T2m&subset=Lat%2852.0,54.00%29&subset=Long%280.0,2.0%29&subset=ansi%28%222012-01-01T00:00:00+00:00%22,%222012-12-31T18:00:00+00:00%22%29 <br><br>

    * or, a specific format encoding can be chosen, e.g. netCDF:    
    
    Example: you add to your GetCoverage request at the end **&format=application/netcdf**<br>
    http://incubator.ecmwf.int/2e/rasdaman/ows?service=WCS&version=2.0.1&request=GetCoverage&coverageId=T2m&subset=Lat%2852.0,54.00%29&subset=Long%280.0,2.0%29&subset=ansi%28%222012-01-01T00:00:00+00:00%22%29&format=application/netcdf 



_____________________________________________________________________________________________________________________ 

### OGC WCS Extension Requests

OGC Web Coverage Processing Service (WCPS) allows extraction, analysis and processing of large gridded geo-referenced datasets.<br> 
At WCS 2.0.1, there is an extension for **PROCESSING**. <br>

With the help of this WCS wrapper for the OGC WCPS standard, complex SQL-based queries can be executed on the coverages.<br>

Queries can be executed by using following request:

In [ ]:
...&request=ProcessCoverages&query=

A query looks as follows (information is encoded as "csv"):

In [ ]:
for c in (parameter) return encode
(c[Lat(double:double),Long(double:double),ansi("2014-01-01T00:00:00+00:00": "2014-01-02T00:00:00+00:00")],"csv")

Keep in mind: the information that can be retrieved depend on the coverage specifications, e.g. number of dimensions. Coverage specifications can be looked up with the **DescribeCoverage** request.

_____________________________________________________________________________________________________________________ 

### How to integrate WCS requests into a python script

In [1]:
# Load required libraries
import requests
import json
import time
import numpy as np
import argparse
import os
import matplotlib.pyplot as plt

WCP Processing query format - Example for 2m air temperature spatial and temporal subsetting and conversion from K to degC

In [9]:
url_fmt = 'http://incubator.ecmwf.int/2e/rasdaman/ows?service=WCS&version=2.0.1' \
          '&request=ProcessCoverages' \
          '&query=for c in (%s) return encode(c[Lat(%f:%f), Long(%f:%f), ansi("%s")] - 273.15, "csv") '

Create your customized url based on the url format above

In [10]:
url = url_fmt % ("T2m", 49.5, 60, 0,5, "2000-12-01T00:00:00+00:00")

Store the data information in a python numpy array for further processing

In [11]:
r= requests.get(url
#            proxies = {"http":"http://proxy.ecmwf.int:3333"}
            )

r.raise_for_status()
x = np.array(eval(r.text.replace('{','[').replace('}',']')))

___________________________________________________

## Practical Examples

### Get Capabilities

Call the **GetCapabilities** documents and examine
* url base path: http://incubator.ecmwf.int/2e/rasdaman/ows
* service: WCS
* version: 2.0.1
* request: GetCapabilities

### Describe Coverage

Get metadata to coverages **Total precipitation** and **discharge_forecast**
* url base path: http://incubator.ecmwf.int/2e/rasdaman/ows
* service: WCS
* version: 2.0.1
* request: DescribeCoverage
* coverageId: **TP** or **discharge_forecast**

### Get Coverage

Retrieve data of **discharge_forecast** from 15 Dec, 2015 | Forecast step: 0 | Ensemble member: 5 for UK
* url base path: http://incubator.ecmwf.int/2e/rasdaman/ows
* service: WCS
* version: 2.0.1
* request: GetCoverage
* coverageId: discharge_forecast
* subset: Lat(48.0, 60.0)
* subset: Long(-11.0, 2.0)
* subset: ansi("2015-12-15T00:00")
* subset: forecast(0)
* subset: ensemble(5)

* format: **application/netcdf** or **image/tiff**

### WCS extension: Process Coverages

Retrieve the same data as from the example above:
* url base path: http://incubator.ecmwf.int/2e/rasdaman/ows
* service: WCS
* version: 2.0.1
* request: ProcessCoverages
* query: for c in (discharge_forecast) return encode (c[Lat(48.0:60.0),Long(-11.0:2.0),ansi("2015-12-15T00:00"),forecast(0),ensemble(0)],"csv")

Retrieve 2m temperature for Italy in the year 2003 and convert it from Kelvin to degC
* url base path: http://incubator.ecmwf.int/2e/rasdaman/ows
* service: WCS
* version: 2.0.1
* request: ProcessCoverages
* query: for c in (T2m) return encode (c[Lat(36.0:46.5),Long(6.0:19.0),ansi("2003-01-01T00:00":"2003-12-31T18:00")] - 273.15,"csv")

Retrieve avg / min / max temperature in Italy in 2003

* ...
* query: for c in (T2m) return encode (max(c[Lat(36.0:46.5),Long(6.0:19.0),ansi("2003-01-01T00:00":"2003-12-31T18:00")]) - 273.15,"csv")
* query: for c in (T2m) return encode (min(c[Lat(36.0:46.5),Long(6.0:19.0),ansi("2003-01-01T00:00":"2003-12-31T18:00")]) - 273.15,"csv")
* query: for c in (T2m) return encode (avg(c[Lat(36.0:46.5),Long(6.0:19.0),ansi("2003-01-01T00:00":"2003-12-31T18:00")]) - 273.15,"csv")

________________________________________

### Enjoy exploring WCS and WCPS!